In [ ]:
# %%
import random as rd
import datetime as dt
import win32com.client as win32
import numpy as np
import pandas as pd
import adodbapi as papi
import os
os.chdir(r"path")
os.getcwd()

# %%
print("imported necessary libraries")

# %%
conn = papi.connect(
    "Provider=MSOLAP.8; Data Source='powerbi://api.powerbi.com/v1.0/myorg/workspace and dataset")
print("connection to service done")
cur1 = conn.cursor()
print("cursor 1 created")
daxMOCD = 'EVALUATE MOCDAGG'
print("evaluated MOCDAGG table")
cur1.execute(daxMOCD)
print("cursor 1 executed")
MOCD = cur1.fetchall()
print("fetched data for cursor 1")
cur2 = conn.cursor()
print("cursor 2 created")
daxRegion = 'EVALUATE Region'
print("evaluated Region table")
cur2.execute(daxRegion)
print("cursor 2 executed")
Region = cur2.fetchall()
print("fetched data for cursor 2")
conn2 = papi.connect(
      "Provider=MSOLAP.8; Data Source='powerbi://api.powerbi.com/v1.0/myorg/workspace and dataset")
print("connection 2 to service done")
cur3 = conn2.cursor()
print("cursor 2 executed")
daxTop = 'EVALUATE aggreg'
print("evaluated aggreg table")
cur3.execute(daxTop)
print("cursor 3 executed")
top500 = cur3.fetchall()
print("fetched data for cursor 3")
cur4 = conn.cursor()
print("cursor 4 executed")
daxTender = 'EVALUATE Tender'
print("evaluated Tender table")
cur4.execute(daxTender)
print("cursor 4 executed")
Tender = cur4.fetchall()
print("fetched data for cursor 4")
print("All data is fetched")


# %%
MOCD = pd.DataFrame(data=MOCD, index=None, columns=[
                    "MC Description", "Region Name", "Sales", "Year", "Month Name", "Month", "Department", "Emirates", "Site Name", "Brand"])
print("created MOCD table")
MOCDp = pd.pivot_table(data=MOCD, values=["Sales"], index="MC Description", columns=[
                       "Region Name", "Year", "Month"], aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total").sort_index(ascending=True)
print("created MOCDp table")
Region = pd.DataFrame(data=Region, index=None, columns=[
                      "Region", "Upsales", "Tenderamount", "Turnover"])
print("created Region table")
Summary1 = pd.pivot_table(data=MOCD, values="Sales",
                          index="Region Name", aggfunc=np.sum)
Summary1.loc["Total"] = Summary1.sum(numeric_only=True)
print("created Summary 1 table")
top500 = pd.DataFrame(data=top500, index=None, columns=[
                      "Material Description", "Material", "Sales Value", "Quantity", "MCD", "BaseUOM", "Tender amt"])
top500 = top500.drop(["Tender amt"], axis=1).sort_values(
    "Sales Value", ascending=False).reset_index(drop=True).head(500)
print("created top500 table")
Tender = pd.DataFrame(data=Tender, index=None, columns=[
                      "Region Name", "Month", "Tender"])
Tenderp = pd.pivot_table(data=Tender, values="Tender", index="Region Name",
                         columns="Month", margins=True, margins_name="Tender Total", aggfunc=np.sum)
print("created tender table")
Totalsalesp = pd.pivot_table(data=MOCD, values="Sales", index="Region Name",
                             columns="Month", aggfunc=np.sum, margins=True, margins_name="Sales Total")
print("created Totalsalesp table")
MOCDDeptp = pd.pivot_table(data=MOCD, values="Sales", index="Department", columns=[
                           "Region Name", "Year", "Month"], aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total").sort_index(ascending=True)
print("created MOCDDept table")
Emirateswisesales = pd.pivot_table(
    data=MOCD, values="Sales", index="Emirates", aggfunc=np.sum).sort_values("Sales", ascending=False)
print("created Emirateswisesales table")
Top50brands = pd.pivot_table(data=MOCD, values="Sales", index="Brand",
                             aggfunc=np.sum).sort_values("Sales", ascending=False)
print("created Top50brands table")
auhstore, dxbstore, alainstore, sharjahstore = MOCD.loc[MOCD["Region Name"] == "Abu Dhabi"], MOCD.loc[MOCD["Region Name"]
                                                                                                      == "Dubai"], MOCD.loc[MOCD["Region Name"] == "Al Ain"], MOCD.loc[MOCD["Region Name"] == "Sharjah"]
top5auh = pd.pivot_table(data=auhstore, index=[
                         "Region Name", "Site Name"], values="Sales", aggfunc=np.sum).sort_values(by="Sales", ascending=False).head()
top5dxb = pd.pivot_table(data=dxbstore, index=[
                         "Region Name", "Site Name"], values="Sales", aggfunc=np.sum).sort_values(by="Sales", ascending=False).head()
top5alain = pd.pivot_table(data=alainstore, index=[
                           "Region Name", "Site Name"], values="Sales", aggfunc=np.sum).sort_values(by="Sales", ascending=False).head()
top5sharjah = pd.pivot_table(data=sharjahstore, index=[
                             "Region Name", "Site Name"], values="Sales", aggfunc=np.sum).sort_values(by="Sales", ascending=False).head()
top5 = pd.concat([top5auh, top5dxb, top5alain, top5sharjah])
print("created top5 table")
#print(top5auh,top5dxb, top5alain,top5sharjah)


# %%
today = dt.date.today().strftime('%d-%m-%Y')
writer = pd.ExcelWriter(f"MOCD {today}.xlsx", engine='xlsxwriter')
Summary1.to_excel(writer, sheet_name='Summary')
Tenderp.to_excel(writer, sheet_name='Summary', startrow=8)
Totalsalesp.to_excel(writer, sheet_name='Summary', startrow=16)
Emirateswisesales.to_excel(writer, sheet_name='Emirates wise sales')
Top50brands.to_excel(writer, sheet_name='Top 50 brands')
top5.to_excel(writer, sheet_name="Top 5 sites by emirates")
MOCDDeptp.to_excel(writer, sheet_name="Dept-MOM")
MOCDp.to_excel(writer, sheet_name='MC-MOM')
Region.to_excel(writer, sheet_name='Region')
top500.to_excel(writer, sheet_name="Top500")
writer.close()
print("created the excel file")


# %%
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.Subject = 'MOCD SSP'

mail.Attachments.Add(os.getcwd()+rf"\\MOCD {today}.xlsx")
mail.To = 'enter to email'
mail.CC = 'enter cc email'
mail.HTMLBody = rf'''<h3>Dear name,</h3>
<br>
PFA MOCD data for {dt.date.today().strftime('%d-%m-%Y')}.<br><br>
Regards,<br>
Fadi
'''
mail.send

# %%
conn.close()
conn2.close()
print("Successful!")

# %%
